In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient 
import dotenv
import requests
from bs4 import BeautifulSoup
import datetime



In [2]:
oai_api_key = dotenv.get_key('.env.local', 'OPENAI_API_KEY')

In [3]:
async def get_abasin_info() -> str:
    f"""Get raw conditions data from Arapahoe Basin's website."""
    response = requests.get('https://www.arapahoebasin.com/snow-report/')
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        condition_divs = soup.find_all('div', class_=['conditions', 'weather'])
        return condition_divs
    return []

In [29]:
from scrape_caic import get_avalanche_info

async def main() -> None:
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    abasin_agent = AssistantAgent(
        name="abasin_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key=oai_api_key,
        ),
        tools=[get_abasin_info, get_avalanche_info(39.6427, -105.8718)],
        system_message=f"""Return a short conditions assessment followed by a markdown 
        summary of A Basin's conditions. If you need avalanche conditions, call the get_avalanche_conditions 
        tool and pass it two variables, the lat 39.6427 and long 105.8718. Let the other agents 
        know this is all the information you can access at this time. The current date is {current_date}.""",
    )

    ski_agent = AssistantAgent(
        "primary",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key=oai_api_key,
        ),
        tools=[],
        system_message=f"""Describe why or why the user should not go skiing at a given time. You can chat with
        another AI agent by calling a tool to get weather and conditions. The current date is {current_date}. 
        
        Anything you say is just your internal thoughts and conversation with the other AI agents. When you have 
        made your decision, you can send a message to the user by including 'TERMINATE' as the final output.""",
    )

    # max_msg_termination = MaxMessageTermination(max_messages=3)
    termination = TextMentionTermination("TERMINATE")
    agent_team = RoundRobinGroupChat([ski_agent, abasin_agent], termination_condition=(termination))

    stream = agent_team.run_stream(task="Should I ski at A Basin tomorrow or hit the backcountry nearby?")
    await Console(stream)

In [ ]:
await main()